Facultad de Ingeniería Industrial y Sistemas - Universidad Nacional de Ingeniería

Alejos Yarasca Fiorella Andrea (fiorella.alejos.y@uni.pe)

Llana Chavez Walter Rodolfo (walter.llana.c@uni.pe)

Medina Rodríguez Henry (hmedinar@uni.pe)

Salazar Vega Edwin Martín (edwin.salazar.v@uni.pe)


Curso: Computación Evolutiva - Maestría en Inteligencia

Artificial 2025-II


In [ ]:
# -------------------------------------------------------------------
# Proyecto de Computación Evolutiva
# Optimización Evolutiva de Asignación de Pedidos (Dataset LADE)
# -------------------------------------------------------------------
# Este proyecto implementa un algoritmo evolutivo multiobjetivo,
# basado en NSGA-II, para abordar el problema de asignación de pedidos
# a repartidores en servicios de entrega bajo demanda.
#
# Se utiliza el dataset LADE (Large-scale Dataset for Delivery),
# específicamente el archivo pickup_yt.csv, que contiene información
# real de pedidos en el punto de recogida, incluyendo datos temporales
# y coordenadas geográficas.
#
# El problema se modela como un problema de asignación, donde cada
# solución candidata representa una posible asignación de pedidos a
# un conjunto fijo de repartidores. No se utiliza la asignación real
# del dataset (courier_id), ya que el objetivo es optimizar dicha
# asignación mediante técnicas evolutivas.
#
# El algoritmo NSGA-II permite optimizar simultáneamente múltiples
# objetivos en conflicto, entre ellos:
#   - Minimizar el tiempo total estimado de atención de los pedidos.
#   - Minimizar la distancia total asociada a los pedidos asignados.
#   - Minimizar el desbalance de carga entre repartidores.
#
# Debido al gran tamaño del dataset LADE, se trabaja con subconjuntos.
#
# Como resultado, el proyecto obtiene un conjunto de soluciones no
# dominadas (frente de Pareto), que representan distintos compromisos
# entre los objetivos considerados, permitiendo analizar alternativas
# eficientes de asignación de pedidos.
# -------------------------------------------------------------------


In [ ]:
# Instala las librerías necesarias para el proyecto:
# - pymoo: implementación de algoritmos evolutivos y multiobjetivo (ej. NSGA-II)
# - pandas: manipulación y análisis de datos tabulares (dataset LADE)
# - numpy: operaciones numéricas y manejo de arreglos
# - matplotlib: visualización de resultados, como el frente de Pareto
!pip install pymoo pandas numpy matplotlib


In [ ]:
#versiones fijadas
!pip install -q datasets==2.21.0 huggingface_hub==0.24.6 pyarrow pandas==2.2.2 fsspec==2024.6.1



In [ ]:
!pip -q install datasets==2.21.0 huggingface_hub==0.24.6 fsspec==2024.6.1 aiohttp


In [ ]:
import os
os.environ["HF_HUB_ETAG_TIMEOUT"] = "120"
os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "600"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"


In [ ]:
from huggingface_hub import snapshot_download

local_dir = snapshot_download(
    repo_id="Cainiao-AI/LaDe",
    repo_type="dataset",
    local_dir="LaDe_local",
    local_dir_use_symlinks=False,
    resume_download=True,
    max_workers=2
)

print("Dataset descargado en:", local_dir)


In [ ]:
#ver con detalle el dataset descaragado
!find /content/LaDe_local -maxdepth 3 -type d


In [ ]:
# ------------------------------------------------------------
# Dataset: LADE (Large-scale Dataset for Delivery)
# ------------------------------------------------------------
# Este proyecto utiliza el dataset LADE, un conjunto de datos
# a gran escala publicado por Cainiao-AI para investigación en
# problemas de logística y servicios de entrega bajo demanda.
#
# El dataset contiene información real de pedidos, incluyendo:
# - Datos de recogida (pickup) en restaurantes o comercios
# - Datos de entrega (delivery) a clientes finales
# - Información de trayectorias de repartidores (opcional)
# - Red vial (road-network) para análisis avanzados
#
# Para este trabajo se utiliza únicamente el archivo:
#   /content/LaDe_local/pickup/pickup_yt.csv
#
# Este archivo contiene los pedidos en el punto de recogida,
# con información temporal y geográfica suficiente para modelar
# el problema de asignación de pedidos a repartidores.
#
# Debido al gran tamaño del dataset completo, se trabaja con
# subconjuntos del archivo (por ventanas de tiempo) para construir
# instancias manejables del problema de optimización evolutiva.
#
# Las demás carpetas del dataset (delivery, road-network y
# data_with_trajectory_20s) no se utilizan en esta implementación,
# ya que exceden el alcance del presente proyecto.
# ------------------------------------------------------------


In [ ]:
import pandas as pd

df = pd.read_csv(
    "/content/LaDe_local/pickup/pickup_yt.csv",
    low_memory=False
)

df.head()


In [ ]:
print("✅ CSV recargado. Filas:", len(df))
df[["accept_time"]].head()

In [ ]:
df.columns


In [ ]:
# -------------------------------------------------------------------
# Columnas del dataset LADE (archivo: pickup_yt.csv)
# -------------------------------------------------------------------
# order_id:
#   Identificador único del pedido.
#   Uso en el proyecto: NO (solo referencia y trazabilidad).
#
# region_id:
#   Identificador de la región o zona dentro de la ciudad.
#   Uso en el proyecto: OPCIONAL (para filtrar pedidos por región).
#
# city:
#   Ciudad donde se realiza el pedido.
#   Uso en el proyecto: OPCIONAL (se puede trabajar con una sola ciudad).
#
# courier_id:
#   Identificador del repartidor que atendió el pedido en el sistema real.
#   Uso en el proyecto: NO (información histórica; la asignación es optimizada).
#
# accept_time:
#   Momento en que un repartidor acepta el pedido.
#   Uso en el proyecto: NO (dato histórico).
#
# time_window_start:
#   Inicio de la ventana de tiempo para la recogida del pedido.
#   Uso en el proyecto: OPCIONAL (restricción temporal en modelos avanzados).
#
# time_window_end:
#   Fin de la ventana de tiempo para la recogida del pedido.
#   Uso en el proyecto: OPCIONAL (restricción temporal en modelos avanzados).
#
# lng:
#   Longitud geográfica del punto de recogida (restaurante).
#   Uso en el proyecto: SÍ (cálculo de distancias).
#
# lat:
#   Latitud geográfica del punto de recogida (restaurante).
#   Uso en el proyecto: SÍ (cálculo de distancias).
#
# aoi_id:
#   Identificador del área de interés (Area of Interest).
#   Uso en el proyecto: OPCIONAL (agrupación espacial o análisis adicional).
#
# aoi_type:
#   Tipo de área (residencial, comercial, etc.).
#   Uso en el proyecto: NO.
#
# pickup_time:
#   Hora real en la que se realizó la recogida del pedido.
#   Uso en el proyecto: NO (resultado histórico).
#
# pickup_gps_time:
#   Timestamp GPS registrado durante la recogida.
#   Uso en el proyecto: NO.
#
# pickup_gps_lng:
#   Longitud GPS registrada durante la recogida.
#   Uso en el proyecto: NO.
#
# pickup_gps_lat:
#   Latitud GPS registrada durante la recogida.
#   Uso en el proyecto: NO.
#
# accept_gps_time:
#   Timestamp GPS registrado cuando el pedido fue aceptado.
#   Uso en el proyecto: NO.
#
# accept_gps_lng:
#   Longitud GPS registrada al aceptar el pedido.
#   Uso en el proyecto: NO.
#
# accept_gps_lat:
#   Latitud GPS registrada al aceptar el pedido.
#   Uso en el proyecto: NO.
#
# ds:
#   Fecha del pedido (formato day stamp).
#   Uso en el proyecto: OPCIONAL (filtrado por día).
# -------------------------------------------------------------------


In [ ]:
import pandas as pd
import numpy as np
import time

t0 = time.time()
print("🔹 Iniciando limpieza y reconstrucción de timestamps...")

# ------------------------------------------------------------
print("➡️ 1) Convirtiendo lat/lng a numérico...")
df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
df["lng"] = pd.to_numeric(df["lng"], errors="coerce")
print("   ✔ lat/lng listos")

# ------------------------------------------------------------
print("➡️ 2) Reconstruyendo accept_time_dt usando ds (MDD) + hora (HH:MM:SS)...")

YEAR = 2020  # Ajusta si tu dataset corresponde a otro año

# ds viene como MDD (ej: 518 -> 05-18). Aseguramos 3 dígitos mínimo.
ds_str = df["ds"].astype("Int64").astype(str).str.zfill(3)  # ej: 518, 505, 511

mm = ds_str.str[:-2].str.zfill(2)   # mes: "5" -> "05"
dd = ds_str.str[-2:]                # día: "18"

# accept_time viene como "MM-DD HH:MM:SS" -> tomamos solo "HH:MM:SS"
time_str = df["accept_time"].astype(str).str[-8:]  # "08:16:00"

# Timestamp final coherente: "YYYY-MM-DD HH:MM:SS"
full_ts = f"{YEAR}-" + mm + "-" + dd + " " + time_str

df["accept_time_dt"] = pd.to_datetime(
    full_ts, format="%Y-%m-%d %H:%M:%S", errors="coerce"
)

ok_dt = df["accept_time_dt"].notna().sum()
print(f"   ✔ accept_time_dt creado | No nulos: {ok_dt}")

# ------------------------------------------------------------
print("➡️ 3) Seleccionando columnas relevantes y eliminando nulos...")
cols_use = ["order_id", "city", "region_id", "accept_time_dt", "lat", "lng", "ds"]

before = len(df)
data = df[cols_use].dropna().copy()
after = len(data)

data = data.rename(columns={"accept_time_dt": "accept_time"})

print(f"   ✔ Filas originales: {before}")
print(f"   ✔ Filas válidas:    {after}")
print(f"   ✔ Eliminadas:       {before - after}")

# ------------------------------------------------------------
t1 = time.time()
print(f"✅ Proceso finalizado en {t1 - t0:.2f} segundos")
print("🔎 Vista previa (data):")
data.head(10)


In [ ]:
# Ordenar los pedidos según el tiempo de aceptación
# Esto asegura que los pedidos sigan un orden temporal realista
data = data.sort_values("accept_time").reset_index(drop=True)

# Seleccionar los primeros 200 pedidos para construir una instancia del problema
# Esta instancia será utilizada en los experimentos del algoritmo evolutivo
instance = data.head(200).reset_index(drop=True)
print("✅ Pedidos en instancia:", len(instance))
instance.head()


In [ ]:
import os

BASE_DIR = "/content/drive/MyDrive/LADE_Evolutionary_Project"
DATA_DIR = os.path.join(BASE_DIR, "data")
RESULTS_DIR = os.path.join(BASE_DIR, "results")

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print("✅ Carpetas creadas")


In [ ]:
PARQUET_PATH = "/content/drive/MyDrive/LADE_Evolutionary_Project/data/LADE_instance_200.parquet"

instance.to_parquet(PARQUET_PATH, index=False)

print(f"✅ Instancia guardada en:\n{PARQUET_PATH}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!ls /content/drive/MyDrive/LADE_Evolutionary_Project/data


In [ ]:

# Ruta
BASE_DIR = "/content/drive/MyDrive/LADE_Evolutionary_Project"
PARQUET_PATH = f"{BASE_DIR}/data/LADE_instance_200.parquet"

instance = pd.read_parquet(PARQUET_PATH)

print("✅ Instancia cargada")
print("Pedidos:", len(instance))
instance.head()


In [ ]:
coords = instance[["lat", "lng"]].to_numpy(dtype=float)

center_lat = coords[:, 0].mean()
center_lng = coords[:, 1].mean()


K = 20
V_KM_H = 20.0
SERVICE_MIN = 3.0

# Función para calcular la distancia geodésica (en kilómetros)
# entre dos puntos dados por latitud y longitud usando la fórmula de Haversine

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1 = np.radians(lat1); lon1 = np.radians(lon1)
    lat2 = np.radians(lat2); lon2 = np.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    return 2 * R * np.arcsin(np.sqrt(a))

# Inicializar generador de números aleatorios con semilla fija
# Esto garantiza la reproducibilidad de los experimentos
# Bases de couriers (K puntos tomados de pedidos reales)
rng = np.random.default_rng(1)
base_idx = rng.choice(len(coords), size=K, replace=False)
bases = coords[base_idx]  # shape (K,2)

# Matriz distancias: dist_matrix[k, j] = distancia base courier k -> pedido j
dist_matrix = np.zeros((K, len(coords)), dtype=float)
for k in range(K):
    dist_matrix[k] = haversine_km(bases[k,0], bases[k,1], coords[:,0], coords[:,1])




In [ ]:
from pymoo.core.problem import Problem

# Definición del problema de asignación de pedidos a repartidores

class AssignOrdersProblem(Problem):
    def __init__(self, dist_matrix, K, v_kmh, service_min):
        """
        Inicializa el problema de optimización.

        Parameters:
        - dist_matrix: matriz de distancias (K x N), donde cada fila representa un repartidor
          y cada columna un pedido.
        - K: número de repartidores.
        - v_kmh: velocidad promedio de desplazamiento (km/h).
        - service_min: tiempo fijo de servicio por pedido (minutos).
        """
        # n_var: número de variables de decisión (una por pedido)
        # n_obj: número de objetivos (makespan, distancia total, balance)
        # xl, xu: límites inferiores y superiores para la asignación de pedidos
        # type_var: las variables son enteras (índice del repartidor)

        super().__init__(n_var=dist_matrix.shape[1], n_obj=3, xl=0, xu=K-1, type_var=int)
        self.dist_matrix = dist_matrix
        self.K = K
        # Conversión de velocidad de km/h a minutos por kilómetro
        self.min_per_km = 60.0 / v_kmh
        # Tiempo fijo de atención por pedido
        self.service_min = service_min

    def _evaluate(self, X, out, *args, **kwargs):
        """
        Evalúa una población de soluciones.

        Parameters:
        - X: matriz de decisiones (pop_size x n_orders),
             donde cada fila representa una solución candidata.
        - out: diccionario donde se almacenan los valores de los objetivos.
        """
        pop_size, n_orders = X.shape

        f1 = np.zeros(pop_size)  # makespan
        f2 = np.zeros(pop_size)  # distancia total
        f3 = np.zeros(pop_size)  # desbalance
        # Evaluar cada solución de la población

        for i in range(pop_size):
            # Asignación de pedidos a repartidores para la solución i
            assign = X[i].astype(int)

            dist_per = np.zeros(self.K, dtype=float)
            cnt_per  = np.zeros(self.K, dtype=int)

            # sumar distancias por courier según asignación
            # Acumular distancias y pedidos según la asignación
            for j in range(n_orders):
                c = assign[j]
                dist_per[c] += self.dist_matrix[c, j]
                cnt_per[c]  += 1

            time_per = dist_per * self.min_per_km + cnt_per * self.service_min

            f1[i] = time_per.max()                # minimizar peor courier
            f2[i] = dist_per.sum()                # minimizar distancia total
            f3[i] = cnt_per.max() - cnt_per.min() # balance

            # penalizar couriers sin pedidos (opcional pero útil)
            empty = np.sum(cnt_per == 0)
            if empty > 0:
                f1[i] += 20.0 * empty
                f3[i] += empty
        # Asignar los valores de los objetivos al diccionario de salida
        out["F"] = np.column_stack([f1, f2, f3])

# Crear la instancia del problema con los parámetros definidos
problem = AssignOrdersProblem(dist_matrix, K=K, v_kmh=V_KM_H, service_min=SERVICE_MIN)

# Mostrar información básica del problema creado

print("✅ Problema creado. Variables:", problem.n_var, "| Objetivos:", problem.n_obj)


In [ ]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
import matplotlib.pyplot as plt

# Configuración del algoritmo NSGA-II
# pop_size define el tamaño de la población en cada generación
algorithm = NSGA2(pop_size=120)

# Ejecución del proceso de optimización evolutiva
# - problem: instancia del problema multiobjetivo definido
# - algorithm: algoritmo evolutivo seleccionado (NSGA-II)
# - termination: criterio de parada basado en número de generaciones
# - seed: semilla para asegurar reproducibilidad
# - verbose: muestra información del progreso en consola
# - save_history: almacena el historial evolutivo para análisis posterior
res = minimize(
    problem,
    algorithm,
    termination=("n_gen", 250),
    seed=1,
    verbose=True,
    save_history=True
)
# Extraer los valores de las funciones objetivo de las soluciones del frente de Pareto
F = res.F
print("✅ Pareto solutions:", F.shape[0])

# Visualización del frente de Pareto
# Ejes:
# - eje X: makespan (f1)
# - eje Y: distancia total (f2)
# El color representa el desbalance de carga (f3)
plt.figure()
sc = plt.scatter(F[:,0], F[:,1], c=F[:,2], s=18)
plt.xlabel("f1: Makespan (min) - max tiempo por courier")
plt.ylabel("f2: Distancia total (km)")
plt.colorbar(sc, label="f3: Desbalance (max-min)")
plt.title("Frente de Pareto - NSGA-II (LADE, 200 pedidos)")
plt.show()


In [ ]:
# Inicializar listas para almacenar la mejor y la aptitud promedio
# de cada función objetivo a lo largo de las generaciones
best_f1, avg_f1 = [], []
best_f2, avg_f2 = [], []
best_f3, avg_f3 = [], []

# Recorrer el historial evolutivo generado por NSGA-II
# Cada elemento de res.history corresponde a una generación
for algo in res.history:
    # objetivos de la población de esa generación
    # Obtener los valores de las funciones objetivo de la población actual
    # Fg tiene dimensión (tamaño_población x número_de_objetivos)
    Fg = algo.pop.get("F")  # shape: (pop_size, 3)

    # f1: makespan
    # Guardar el mejor valor (mínimo) y el valor promedio de la población

    best_f1.append(Fg[:, 0].min())
    avg_f1.append(Fg[:, 0].mean())

    # f2: distancia total
    # Guardar el mejor valor (mínimo) y el valor promedio de la población

    best_f2.append(Fg[:, 1].min())
    avg_f2.append(Fg[:, 1].mean())

    # f3: desbalance de carga
    # Guardar el mejor valor (mínimo) y el valor promedio de la población

    best_f3.append(Fg[:, 2].min())
    avg_f3.append(Fg[:, 2].mean())

best_f1 = np.array(best_f1); avg_f1 = np.array(avg_f1)
best_f2 = np.array(best_f2); avg_f2 = np.array(avg_f2)
best_f3 = np.array(best_f3); avg_f3 = np.array(avg_f3)

# Mostrar el número total de generaciones registradas
print("Generaciones registradas:", len(best_f1))


Grafica Convergencia

In [ ]:
plt.figure()
plt.plot(best_f1, label="Mejor f1 (min)")
plt.plot(avg_f1, label="Promedio f1")
plt.xlabel("Generación")
plt.ylabel("Makespan (min)")
plt.title("Convergencia de f1 (Makespan)")
plt.legend()
plt.show()


Grafico convergencia - Ditancia total

In [ ]:
plt.figure()
plt.plot(best_f2, label="Mejor f2 (min)")
plt.plot(avg_f2, label="Promedio f2")
plt.xlabel("Generación")
plt.ylabel("Distancia total (km)")
plt.title("Convergencia de f2 (Distancia total)")
plt.legend()
plt.show()


In [ ]:
plt.figure()
plt.plot(best_f3, label="Mejor f3 (min)")
plt.plot(avg_f3, label="Promedio f3")
plt.xlabel("Generación")
plt.ylabel("Desbalance (max-min)")
plt.title("Convergencia de f3 (Desbalance)")
plt.legend()
plt.show()


In [ ]:
window = 30
delta_f1 = best_f1[-window] - best_f1[-1]
delta_f2 = best_f2[-window] - best_f2[-1]
delta_f3 = best_f3[-window] - best_f3[-1]

print(f"Mejora en best f1 (últimas {window} gens): {delta_f1:.4f}")
print(f"Mejora en best f2 (últimas {window} gens): {delta_f2:.4f}")
print(f"Mejora en best f3 (últimas {window} gens): {delta_f3:.4f}")


Grafica convergencia - desbalance

In [ ]:
F = res.F.copy()

# normalizar a [0,1] cada objetivo
F_norm = (F - F.min(axis=0)) / (F.max(axis=0) - F.min(axis=0) + 1e-12)

# score por distancia al origen (0,0,0): menor = mejor compromiso
score = np.linalg.norm(F_norm, axis=1)

best_idx = np.argmin(score)
print("✅ Índice solución compromiso:", best_idx)
print("f1,f2,f3:", res.F[best_idx])


In [ ]:
best_assign = res.X[best_idx].astype(int)
instance_solution = instance.copy()
instance_solution["courier_assigned"] = best_assign

instance_solution.head()



In [ ]:
counts = instance_solution["courier_assigned"].value_counts().sort_index()
print("Pedidos por courier (min/max):", counts.min(), counts.max())
counts.head()


In [ ]:
out_path = f"{RESULTS_DIR}/best_solution_orders.csv"
instance_solution.to_csv(out_path, index=False)
print("✅ Guardado:", out_path)


In [ ]:
df = pd.read_csv(f"{RESULTS_DIR}/best_solution_orders.csv")
tabla_asignacion = (
    df.groupby("courier_assigned")
      .size()
      .reset_index(name="num_pedidos")
      .sort_values("courier_assigned")
)

tabla_asignacion


In [ ]:
tabla_asignacion.to_csv(
    "tabla_asignacion_couriers.csv",
    index=False
)

In [ ]:
min_pedidos = tabla_asignacion["num_pedidos"].min()
max_pedidos = tabla_asignacion["num_pedidos"].max()
balance = max_pedidos - min_pedidos

min_pedidos, max_pedidos, balance


Estudio de sensibilidad

In [ ]:
import time
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize

def run_nsga(pop_size, n_gen=250):
    algorithm = NSGA2(pop_size=pop_size)

    start = time.time()
    res = minimize(
        problem,
        algorithm,
        termination=("n_gen", n_gen),
        seed=1,
        verbose=False
    )
    elapsed = time.time() - start

    F = res.F

    return {
        "pop_size": pop_size,
        "best_f1": F[:,0].min(),
        "best_f2": F[:,1].min(),
        "best_f3": F[:,2].min(),
        "pareto_size": F.shape[0],
        "time_sec": elapsed
    }


In [ ]:
results = []
for p in [60, 120, 200]:
    results.append(run_nsga(p))

df_sens = pd.DataFrame(results)
df_sens


In [ ]:

def random_assignment(dist_matrix, K, v_kmh, service_min, seed=1):
    np.random.seed(seed)

    n_orders = dist_matrix.shape[1]
    assign = np.random.randint(0, K, size=n_orders)

    dist_per = np.zeros(K)
    cnt_per = np.zeros(K, dtype=int)

    for j in range(n_orders):
        c = assign[j]
        dist_per[c] += dist_matrix[c, j]
        cnt_per[c] += 1

    min_per_km = 60.0 / v_kmh
    time_per = dist_per * min_per_km + cnt_per * service_min

    f1 = time_per.max()
    f2 = dist_per.sum()
    f3 = cnt_per.max() - cnt_per.min()

    return f1, f2, f3


In [ ]:
f1_rand, f2_rand, f3_rand = random_assignment(
    dist_matrix, K, V_KM_H, SERVICE_MIN
)

print("Baseline aleatorio")
print("f1:", f1_rand)
print("f2:", f2_rand)
print("f3:", f3_rand)
